In [25]:
%%capture
import json
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
import numpy as np
import scipy as sp
from tqdm.notebook import tqdm

In [26]:
RESULTS_DIR_PATH = './results/'
TEST_SET_SIZE = 2000

In [3]:
DATASET_NAME = 'aan'
FULL_SET_PATH = './data/aan_full.json'
RAW_OUTPUT_PATH = './results/raw_results_aan.json'
FILTERED_OUTPUT_PATH = './results/filtered_results_aan.json'

In [27]:
DATASET_NAME = 'dblp'
FULL_SET_PATH = './data/dblp_full.json'
RAW_OUTPUT_PATH = './results/raw_results_dblp.json'
FILTERED_OUTPUT_PATH = './results/filtered_results_dblp.json'

In [28]:
with open(FULL_SET_PATH) as f:
    full_set = json.load(f)
    
full_set_dict = dict([(ref['id'], ref) for ref in full_set])

In [29]:
results = {}
result_files = [f for f in os.listdir(RESULTS_DIR_PATH) if '.json' in f and 'results' not in f and DATASET_NAME in f]

for filename in result_files:
    method_name = filename.split('_' + DATASET_NAME)[0]
    if method_name in results:
        with open(RESULTS_DIR_PATH + filename) as file:
            results[method_name]['recs'] = json.load(file)
    else:
        result = {}
        result['method_name'] = method_name
    
        with open(RESULTS_DIR_PATH + filename) as file:
            result['recs'] = json.load(file)
        
        results[method_name] = result

In [30]:
corpus = [ref['title'] + ' ' + ref['abstract'] for ref in full_set]
vectorizer = TfidfVectorizer(stop_words='english').fit(corpus)

In [31]:
for ref in tqdm(full_set):
    document = [ref['title'] + ' ' + ref['abstract']]
    ref['tfidf_vector'] = vectorizer.transform(document)

# Accuracy

### Content similarity with user profil

In [32]:
for result in tqdm(results.values()):
    scores = []
    
    for rec in result['recs']:
        if rec['output']:
            input_document = []
            for paper_id in rec['input']:
                input_document.append(full_set_dict[paper_id]['title'] + ' ' + full_set_dict[paper_id]['abstract'])
            input_document = ' '.join(input_document)
            input_tfidf_array = vectorizer.transform([input_document])

            output_tfidf_vectors = [full_set_dict[paper_id]['tfidf_vector'] for paper_id in rec['output']]
            output_tfidf_array = sp.sparse.vstack(output_tfidf_vectors)

            score = np.mean(cosine_similarity(input_tfidf_array,output_tfidf_array))        

            scores.append(score)
        else:
            scores.append(0)
        
    result['accuracy_content_tfidf'] = scores

### Graph similarity with user profil

In [33]:
for result in tqdm(results.values()):
    scores = []
    
    for rec in result['recs']:
        if rec['output']:
            input_ref_set = set()
            for paper_id in rec['input']:
                input_ref_set.update(full_set_dict[paper_id]['references'])
                input_ref_set.update(full_set_dict[paper_id]['citations'])

            partial_scores = []
            for paper_id in rec['output']:
                ref_set = set()
                ref_set.update(full_set_dict[paper_id]['references'])
                ref_set.update(full_set_dict[paper_id]['citations'])

                intersection = input_ref_set.intersection(ref_set)
                union = input_ref_set.union(ref_set)

                partial_scores.append(len(intersection)/len(union))

            score = np.mean(partial_scores)
            scores.append(score)
        else:
            scores.append(0)
        
    result['accuracy_graph_jaccard'] = scores

# Coverage

### Global items coverage

In [34]:
for result in tqdm(results.values()):
    rec_ids_set = set()
    for rec in result['recs']:
        rec_ids_set.update(rec['output'])
        
    result['coverage_item_global'] = len(rec_ids_set) / len(full_set)

### Global users coverage

In [35]:
for result in tqdm(results.values()):
    scores = {}
    scores['n_complete'] = len([_ for rec in result['recs'] if len(rec['output']) == 100])
    scores['n_empty'] = len([_ for rec in result['recs'] if not rec['output']])
    scores['n_incomplete'] = len(result['recs']) - scores['n_complete'] - scores['n_empty']
        
    result['coverage_user_global'] = scores

# Diversity

### Infra list content dissimilarity

In [36]:
for result in tqdm(results.values()):
    scores = []
    
    for rec in result['recs']:
        if rec['output'] and len(rec['output']) > 1:
            partial_scores = []

            for current_paper_id in rec['output']:            
                current_tfidf_array = full_set_dict[current_paper_id]['tfidf_vector']
                other_tfidf_vectors = [full_set_dict[paper_id]['tfidf_vector'] for paper_id in rec['output'] if paper_id != current_paper_id]
                other_tfidf_array = sp.sparse.vstack(other_tfidf_vectors)
                partial_scores.append(np.mean(cosine_distances(current_tfidf_array,other_tfidf_array)))

            score = np.mean(partial_scores)
            scores.append(score)
        else:
            scores.append(0)
        
    result['diversity_content_tfidf'] = scores

### Infra list graph dissimilarity

In [37]:
for result in tqdm(results.values()):
    scores = []
    
    for rec in result['recs']:
        if rec['output'] and len(rec['output']) > 1:
            partial_scores = []

            for current_paper_id in rec['output']:
                current_ref_set = set()
                current_ref_set.update(full_set_dict[current_paper_id]['references'])
                current_ref_set.update(full_set_dict[current_paper_id]['citations'])

                for other_paper_id in filter(lambda e: e != current_paper_id, rec['output']):
                    other_ref_set = set()
                    other_ref_set.update(full_set_dict[other_paper_id]['references'])
                    other_ref_set.update(full_set_dict[other_paper_id]['citations'])

                    intersection = input_ref_set.intersection(other_ref_set)
                    union = input_ref_set.union(other_ref_set)

                    partial_scores.append(1 - len(intersection)/len(union))

            score = np.mean(partial_scores)
            scores.append(score)
        else:
            scores.append(0)
        
    result['diversity_graph_jaccard'] = scores

### Global diversity

In [38]:
for result in tqdm(results.values()):
    freq_recs = dict([(paper['id'], 0) for paper in full_set])
    
    for rec in result['recs']:
        for paper_id in rec['output']:
            freq_recs[paper_id] += 1
            
    total_recs = sum(freq_recs.values())
    
    p_recs = [freq / total_recs for freq in freq_recs.values() if freq > 0]
        
    result['diversity_global'] = -1 * sum([p_rec * np.log2(p_rec) for p_rec in p_recs])

# Novelty

### Average publication year

In [39]:
for result in tqdm(results.values()):
    scores = []
    
    for rec in result['recs']:
        if rec['output']:
            partial_scores = []
            for paper_id in rec['output']:
                partial_scores.append(full_set_dict[paper_id]['year'])

            score = np.mean(partial_scores)
            scores.append(score)
        else:
            scores.append(0)
        
    result['novelty_average_pub_year'] = scores

### Inverse popularity (approx. by citations)

In [40]:
for result in tqdm(results.values()):
    scores = []
    
    for rec in result['recs']:
        if rec['output']:
            score = []
            for paper_id in rec['output']:
                n_citations = len(full_set_dict[paper_id]['citations'])
                score.append(np.log2((n_citations+1) / len(full_set)))
            scores.append(np.mean(score) * -1)
        else:
            scores.append(0)
        
    result['novelty_inverse_popularity'] = scores

# generate results file

### raw results

In [41]:
for result in results.values():
    result.pop('recs')

In [42]:
with open(RAW_OUTPUT_PATH, 'w') as f:
    json.dump(results, f)

### filtered results

In [43]:
invalid_recs_pos = set()

for filename in result_files:
    with open(RESULTS_DIR_PATH + filename) as file:
        for i, rec in enumerate(json.load(file)):
            if not rec['output'] or len(rec['output']) < 50:
                invalid_recs_pos.add(i)
            
valid_recs_pos = [i for i in range(TEST_SET_SIZE) if i not in invalid_recs_pos]

In [44]:
filtered_results = {}

for method_name, result in results.items():
    filtered_result = {}
    filtered_result['method_name'] = method_name
    filtered_result['coverage_item_global'] = result['coverage_item_global']
    filtered_result['coverage_user_global'] = result['coverage_user_global']
    filtered_result['diversity_global'] = result['diversity_global']
    
    filtered_result['accuracy_content_tfidf'] = [result['accuracy_content_tfidf'][i] for i in valid_recs_pos]
    filtered_result['accuracy_graph_jaccard'] = [result['accuracy_graph_jaccard'][i] for i in valid_recs_pos]
    filtered_result['novelty_average_pub_year'] = [result['novelty_average_pub_year'][i] for i in valid_recs_pos]
    filtered_result['novelty_inverse_popularity'] = [result['novelty_inverse_popularity'][i] for i in valid_recs_pos]
    filtered_result['diversity_graph_jaccard'] = [result['diversity_graph_jaccard'][i] for i in valid_recs_pos]
    filtered_result['diversity_content_tfidf'] = [result['diversity_content_tfidf'][i] for i in valid_recs_pos]
    
    filtered_results[method_name] = filtered_result

In [45]:
with open(FILTERED_OUTPUT_PATH, 'w') as f:
    json.dump(filtered_results, f)